# Run: Human Set 1


## Introduction

```
Lead     : ababaian / jtalyor
Issue    : 
Version  : v0.3.2
start    : 2020 05 28
complete : YYYY MM DD
files    : ~/serratus/notebook/200530_ab/
s3_files : s3://serratus-public/notebook/200530_ab/
output   : s3://serratus-public/out/200530_hu1/
```

### Objectives
- Search all public human metagenome / metatranscriptome SRA accessions
- Search 105K human RNAseq, random choice


## SRA Accession Initialization



### Public human metagenome / metatranscriptomes

SRA Accessed: 2020/05/30
Search Term: 
```
"txid9606"[Organism:exp] AND ("metagenomic"[Filter] OR "metatranscriptomic"[Filter]) AND cluster_public[prop] AND "platform illumina"[Properties]
```

Results: `36052` Accessions saved in `hu1_meta_SraRunInfo.csv`

### Public human RNA-seq

SRA Accessed: 2020/05/30
Search Term: 
```
"txid9606"[Organism:exp] AND ("type_rnaseq"[Filter]) AND cluster_public[prop] AND "platform illumina"[Properties]
```

Results: `572215` Accessions saved in `hu0_SraRunInfo.csv`

In [ ]:
cd /home/artem/serratus/notebook/200530_ab/

# Randomly select 100,000 accessions of human RNA-seq
head -n1 hu0_SraRunInfo.csv > sra.header.tmp

tail -n+2 hu0_SraRunInfo.csv \
  | shuf - \
  | head -n 100000 > hu1.tmp
  
cat sra.header.tmp hu1.tmp > hu1_SraRunInfo.csv

rm *tmp

In [1]:
cd /home/artem/serratus/notebook/200530_ab/
wc -l *
md5sum *
md5sum * > hu1_SraRunInfo.md5sum

aws s3 sync ./ s3://serratus-public/out/200530_hu1/

   672657 hu0_SraRunInfo.csv
   100001 hu1_SraRunInfo.csv
    36104 hu1_meta_SraRunInfo.csv
   808762 total
2d2998b585f6b5035b051b0960692c96  hu0_SraRunInfo.csv
58b5fb088d28735000aec4921bbedb20  hu1_SraRunInfo.csv
8224e6cea6afe2d4da73c23d5804ddd4  hu1_meta_SraRunInfo.csv
upload: ./hu1_SraRunInfo.md5sum to s3://serratus-public/out/200530_hu1/hu1_SraRunInfo.md5sum
upload: ./hu1_meta_SraRunInfo.csv to s3://serratus-public/out/200530_hu1/hu1_meta_SraRunInfo.csv
upload: ./hu1_SraRunInfo.csv to s3://serratus-public/out/200530_hu1/hu1_SraRunInfo.csv
upload: ./hu0_SraRunInfo.csv to s3://serratus-public/out/200530_hu1/hu0_SraRunInfo.csv


In [3]:
cd /home/artem/serratus/notebook/200530_ab/

# Randomize meta-genomics input
head -n1 hu1_meta_SraRunInfo.csv > sra.header.tmp

tail -n+2 hu1_meta_SraRunInfo.csv \
  | shuf - \
  > sra.tmp

cat sra.header.tmp sra.tmp > hu1_meta_rand_SraRunInfo.csv

rm *tmp

In [2]:
date

Sat May 30 13:32:22 PDT 2020


## Run Batch 2


### Initialize local workspace

In [8]:
# Serratus commit version
SERRATUS="/home/artem/serratus"
cd $SERRATUS
git rev-parse HEAD # commit version

503789158470f3fd45badd35e5540a51d360057d


In [6]:
# Create local run directory
WORK="$SERRATUS/notebook/200530_ab"
mkdir -p $WORK; cd $WORK

# SRA RunInfo Table base for run
RUNINFO="$WORK/hu1_meta_rand_SraRunInfo.csv"

md5sum $RUNINFO
aws s3 cp $RUNINFO s3://serratus-public/out/200530_hu1/

668aab3b874643143c2326e708604a01  /home/artem/serratus/notebook/200530_ab/hu1_meta_rand_SraRunInfo.csv
upload: ./hu1_meta_rand_SraRunInfo.csv to s3://serratus-public/out/200530_hu1/hu1_meta_rand_SraRunInfo.csv


# Run - Human Metagenomics

In [7]:
cd $WORK
CURRENT_BATCH=$WORK/hu1_meta_rand_SraRunInfo.csv
echo $CURRENT_BATCH

/home/artem/serratus/notebook/200530_ab/hu1_meta_rand_SraRunInfo.csv


In [9]:
# Terraform customization
git diff $SERRATUS/terraform/main/main.tf

In [ ]:
# Load SRA Run Info into scheduler ===================
# Scheduler DNS: 
echo "Loading SRARunInfo into scheduler "
echo "  File: $CURRENT_BATCH"
echo "  md5 : $(md5sum $CURRENT_BATCH)"
echo "  date: $(date)"

curl -s -X POST -T $CURRENT_BATCH localhost:8000/jobs/add_sra_run_info/

In [ ]:
# Set Cluster Parameters =============================
## get Config File (if it doesn't exist)
# curl localhost:8000/config | jq > serratus-config.json
#
cd $TF
# Make local changes to config file
echo "  Cluster Config File: "
cat serratus-config.json
echo ""
echo ""
# Re-upload config file
curl -T serratus-config.json localhost:8000/config

## Update Error handling / controls


In [ ]:
## Stop postgres if it's running 
# systemctl stop postgresql

## Connect to postgres
# psql -h localhost postgres postgres

### ACCESSION OPERATIONS
## Reset SPLITTING accessions to NEW
# UPDATE acc SET state = 'new' WHERE state = 'splitting';

## Reset SPLIT_ERR accessions to NEW
## (repeated failures can be missing SRA data)
# UPDATE acc SET state = 'new' WHERE state = 'split_err';

## Reset MERGE_ERR accessions to SPLIT_DONE
# UPDATE acc SET state = 'split_done' WHERE state = 'merge_err';

## Clear DONE Accessions (ONLY ON COMPLETION)
# DELETE FROM acc WHERE state = 'merge_done';

### BLOCK OPERATIONS

##  Reset FAIL blocks to NEW
# UPDATE blocks SET state = 'new' WHERE state = 'fail';

# Reset ALIGNING blocks to NEW
# UPDATE blocks SET state = 'new' WHERE state = 'aligning';

# Clear DONE blocks
# DELETE FROM blocks WHERE state = 'done';


Time - Merge_done
```
15:10 - 0
16:10 - 1500
17:10 - 20300
18:10 - 47900
19:10 - 60100
20:10 - 71600
21:10 - 84400
22:06 - 100000
22:10 - 102100
22:47 - 132800
```